In [ ]:
from snappy import *
import pandas as pd
import re
import numpy as np


In [ ]:
#preliminary filtering based on determinant and linking matrix

#initialize link table
link_iterator = HTLinkExteriors(knots_vs_links='links')

link_dct = {}
keys = ['name_unoriented', 'determinant', 'num_crossings', 'matrix_zero', 'components', 'jones_factorable', 'jones_determinant', 'smooth_four_genus']

for link in link_iterator:
	linkname = link.name()
	L = link.link()

	num_crossings = int(re.findall('L(\d+)[an]\d+', linkname)[0])

	if L.determinant() == 0 and 12 <= num_crossings:
		print(linkname)
		linking_matrix = L.linking_matrix()
		matrix_zero = all(x == 0 for row in linking_matrix for x in row)
		if matrix_zero:
			#append empty dct with relevant information
			link_dct[linkname] = {key: None for key in keys}

			num_crossings = int(re.findall('L(\d+)[an]\d+', linkname)[0])

			link_dct[linkname]['num_crossings'] = num_crossings
			link_dct[linkname]['determinant'] = 0
			link_dct[linkname]['matrix_zero'] = True
			link_dct[linkname]['name_unoriented'] = linkname
			link_dct[linkname]['components'] = link.num_cusps()


link_df = pd.DataFrame.from_dict(link_dct, orient='index')


<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
/var/folders/lq/dt070ydd3h13pqgrgy6rpv300000gn/T/ipykernel_3010/1156405529.py:13: SyntaxWarning: invalid escape sequence '\d'
  num_crossings = int(re.findall('L(\d+)[an]\d+', linkname)[0])
/var/folders/lq/dt070ydd3h13pqgrgy6rpv300000gn/T/ipykernel_3010/1156405529.py:23: SyntaxWarning: invalid escape sequence '\d'
  num_crossings = int(re.findall('L(\d+)[an]\d+', linkname)[0])
/var/folders/lq/dt070ydd3h13pqgrgy6rpv300000gn/T/ipykernel_3010/1156405529.py:13: SyntaxWarning: invalid escape sequence '\d'
  num_crossings = int(re.findall('L(\d+)[an]\d+', linkname)[0])
/var/folders/lq/dt070ydd3h13pqgrgy6rpv300000gn/T/ipykernel_3010/1156405529.py:23: SyntaxWarning: invalid escape sequence '\d'
  num_crossings = int(re.findall('L(\d+)[an]\d+', linkname)[0])


SageNotAvailable: Sorry, this feature requires using SnapPy inside Sage.

In [ ]:
def factor_jones(row):
	name = row['name_unoriented']
	num_components = row['components']
	R.<q> = QQ[]
	l = Link(name)
	numerator = l.jones_polynomial()
	denominator = (q + q^(-1))^(num_components - 1)
	try:
		factored = numerator.quo_rem(denominator)
		row['jones_factorable'] = True
		#calculate jones determinant by evaluating the factored polynomial at the imaginary number i
		row['jones_determinant'] = factored[0](q=I)
	except:
		row['jones_factorable'] = False
		row['jones_determinant'] = np.nan
	return row

link_df = link_df.apply(factor_jones, axis=1)


In [ ]:
#deconnect link into components. test if the product of the determinants of each component equals the jones determinant modulo 32

shortlist = pd.read_csv('/Users/seohyeonlee/knotinfo/results/potential_ribbons_12_to_14.csv')

#modEight = IntegerModRing(8)

def check_mod(row):
	#modulo eight
	det = row['jones_determinant']
	return mod(det, 8) == 1

manual_check_needed = []
def check_products(row):
	jones_det = row['jones_determinant']
	num_components = row['components']
	linkname = row['name_unoriented']
	link = Link(linkname)
	det_product = 1
	for i in range(num_components):
		sub = link.sublink([i])
		#if not unknot
		try:
			subdet = sub.determinant()
			det_product *= subdet

		except (IndexError, ValueError) as e:
			#print(f"Failed determinant for component {i}: {e}")
			#if unknot, not an issue, pass
			if sub.exterior().identify() == []:
				print(f'component {i} of link {linkname} is an unknot')
				row['has_unknot'] = True
			else:
				print(f'check for component {i} of link {linkname} failed for some other reason')
				manual_check_needed.append((linkname, i))
	row['has_unknot'] = False

	print(linkname, mod(det_product, 32) == mod(jones_det, 32))
	#row['mod32_equivalent'] = (mod(det_product, 32) == mod(jones_det, 32))
	return mod(det_product, 32) == mod(jones_det, 32)


shortlist2.apply(check_products, axis=1)


In [ ]:
for linkname, component_id in manual_check_needed:
	iden = Link(linkname).sublink([component_id]).exterior().identify()
	#Link(link).view()
	sublink_det = Manifold(iden[0]).link().determinant()

	mod(shortlist2.loc[linkname]['jones_determinant'], 32) == mod()


In [ ]:

shortlist = pd.read_csv('/Users/seohyeonlee/knotinfo/results/potential_ribbons_12_to_14.csv')


shortlist['jones_determinant'].max()

shortlist.loc[shortlist['jones_determinant'].idxmax()]


maxjones =  check_products(shortlist.loc[shortlist['jones_determinant'].idxmax()])


Unnamed: 0           L14n12116
name_unoriented      L14n12116
determinant                  0
num_crossings               14
matrix_zero               True
components                   2
jones_factorable          True
jones_determinant          257
smooth_four_genus          NaN
Name: 181, dtype: object

In [ ]:
!sage "link_table_analysis.sage"


component 0 of link L12n84 is an unknot
L12n84 True
component 0 of link L12n399 is an unknot
L12n399 True
component 0 of link L12n418 is an unknot
L12n418 True
component 0 of link L12n467 is an unknot
L12n467 False
component 0 of link L12n508 is an unknot
L12n508 False
component 0 of link L12n520 is an unknot
L12n520 True
component 0 of link L12n529 is an unknot
L12n529 True
component 0 of link L12n544 is an unknot
L12n544 False
component 0 of link L12n545 is an unknot
L12n545 False
component 0 of link L12n564 is an unknot
L12n564 False
component 0 of link L12n817 is an unknot
L12n817 True
component 0 of link L12n910 is an unknot
L12n910 True
component 0 of link L12n1040 is an unknot
L12n1040 False
component 0 of link L12n1095 is an unknot
L12n1095 False
component 0 of link L12n1109 is an unknot
component 1 of link L12n1109 is an unknot
L12n1109 False
component 0 of link L12n1180 is an unknot
component 1 of link L12n1180 is an unknot
L12n1180 True
component 0 of link L12n1181 is an unk

In [ ]:
mod32_applied = pd.read_csv('results/mod32_applied.csv')


In [ ]:
mod32_applied.shape

mod32_applied.head()


,Unnamed: 0.1,Unnamed: 0,name_unoriented,determinant,num_crossings,matrix_zero,components,jones_factorable,jones_determinant,smooth_four_genus,mod32_equivalent
0,0,L12n84,L12n84,0,12,True,2,True,-23,NaN,True
1,1,L12n399,L12n399,0,12,True,2,True,-71,NaN,True
2,2,L12n418,L12n418,0,12,True,2,True,-71,NaN,True
3,5,L12n520,L12n520,0,12,True,2,True,-71,NaN,True
4,6,L12n529,L12n529,0,12,True,2,True,-71,NaN,True


In [ ]:
!sage "fox_milnor_test.sage"


True
True
True
True
True
True
True
True
True
False
False
True
False
False
True
True
True
True
True
True
False
True
True
False
False
False
False
False
True
False
False
True
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
False
True
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
False
True
False
False
True
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
F

In [ ]:
fm = pd.read_csv('results/fox_milnor_applied.csv')

fm.shape


(92, 13)